In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [2]:
model_local = ChatOllama(model="mistral",temperature=0)

In [5]:
loader = BSHTMLLoader(r"C:\Users\Barani\Desktop\local_ollama\test\test_9.html",open_encoding='utf-8')
data = loader.load()
print(data)

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n   Contact Algemeen  | LUMC\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Skip to main content\n   \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n             NL\n            \n\n             /\n            \n\n             EN\n            \n\n\n\n\n\n\n\n\n\n\n\n\n            mijnLUMC\n           \n\n            mijnLUMC\n           \n\n\n\n\n\n\n\n\n             Menu\n            \n\n\n\n\n\n\n\n             Sluiten\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n       Gedownload op 17 maart 2024\n      \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n           Ga terug\n          \n\n           Patiëntenzorg\n          \n\n           Onderwijs\n          \n\n           Research\n          \n\n           Over het LUMC\n          \n\n\n\n\n\n\n              Afdelingen\n             \n\n\n\n\n\n              Contact, route en bezoektijden\n             \n\n\n\n\n\n         

In [15]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=200)
doc_splits = text_splitter.split_documents(data)
# len(doc_splits)

Created a chunk of size 16803, which is longer than the specified 7500
Created a chunk of size 10873, which is longer than the specified 7500
Created a chunk of size 25747, which is longer than the specified 7500
Created a chunk of size 62502, which is longer than the specified 7500
Created a chunk of size 18729, which is longer than the specified 7500
Created a chunk of size 86418, which is longer than the specified 7500
Created a chunk of size 62897, which is longer than the specified 7500
Created a chunk of size 31533, which is longer than the specified 7500
Created a chunk of size 13323, which is longer than the specified 7500
Created a chunk of size 8938, which is longer than the specified 7500
Created a chunk of size 75041, which is longer than the specified 7500
Created a chunk of size 75750, which is longer than the specified 7500
Created a chunk of size 8034, which is longer than the specified 7500
Created a chunk of size 7835, which is longer than the specified 7500
Created a

In [16]:
len(doc_splits)

5736

In [17]:
from langchain_community.vectorstores import FAISS

In [21]:
embeddings = embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text')
db = FAISS.from_documents(doc_splits, embeddings)
print(db.index.ntotal)
db.save_local("cancer_index")

5736


In [22]:
query = "what is cancer"
docs = db.similarity_search(query)
print(docs[0].page_content)

Contact Us

Live Chat


1-800-4-CANCER


NCIinfo@nih.gov


Site Feedback

Follow us


Facebook


Follow on X

Instagram

Youtube


Linkedin


U.S. Department of Health and Human Services
                
National Institutes of Health
                
National Cancer Institute 
USA.gov

https://www.cancer.gov/Common/PopUps/popDefinition.aspx?id=45078&version=patient&language=English&dictionary=Cancer.gov

      Skip to main content
    

        An official website of the United States government
      
Español

Menu


						Search
					


Search

About Cancer


Cancer Types


Research


Grants & Training


News & Events


About NCI

Home


Publications


NCI Dictionaries

NCI Dictionary of Cancer Terms

Publications


Patient Education Publications


PDQ® 


Fact Sheets


NCI Dictionaries


Dictionary of Cancer Terms


Drug Dictionary


Dictionary of Genetics Terms


Blogs and Newsletters


Health Communications Publications


Reports


You need to enable JavaScript to run this app.

B

In [9]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
print(docs[0].page_content)

Contact
Number


General number
507-284-2511


Appointment Office
507-538-3270


Insurance and billing


844-217-9591 (toll free)
507-266-0909 for international callers


International Appointment Office
507-284-8884


International Appointment Office fax
507-538-7802


Medical records and imaging requests
507-284-4594


Medical records requests fax
507-284-0161

Records request form (PDF)
Mayo Clinic Hospital, Methodist Campus
201 W. Center St.
  Rochester, MN 55902

Contact
Number


General number
507-266-7890

Mayo Clinic Hospital, Saint Marys Campus
including Mayo Eugenio Litta Children's Hospital
1216 Second St. SW
  Rochester, MN 55902

Contact
Number


General number
507-255-5123


Rochester, Minnesota, Office of Patient Experience
844-544-0036 (toll-free)


Back to topStaff phone and email directoriesMayo Clinic doesn't have a public directory of staff phone numbers or email addresses. To contact someone, call the general telephone number at a Mayo Clinic location and the opera

In [10]:
db.save_local("faiss_index")

In [12]:
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

new_docs = new_db.similarity_search(query)

In [13]:
docs[0]

Document(page_content="Contact\nNumber\n\n\nGeneral number\n507-284-2511\n\n\nAppointment Office\n507-538-3270\n\n\nInsurance and billing\n\n\n844-217-9591 (toll free)\n507-266-0909 for international callers\n\n\nInternational Appointment Office\n507-284-8884\n\n\nInternational Appointment Office fax\n507-538-7802\n\n\nMedical records and imaging requests\n507-284-4594\n\n\nMedical records requests fax\n507-284-0161\n\nRecords request form (PDF)\nMayo Clinic Hospital, Methodist Campus\n201 W. Center St.\n  Rochester, MN 55902\n\nContact\nNumber\n\n\nGeneral number\n507-266-7890\n\nMayo Clinic Hospital, Saint Marys Campus\nincluding Mayo Eugenio Litta Children's Hospital\n1216 Second St. SW\n  Rochester, MN 55902\n\nContact\nNumber\n\n\nGeneral number\n507-255-5123\n\n\nRochester, Minnesota, Office of Patient Experience\n844-544-0036 (toll-free)\n\n\nBack to topStaff phone and email directoriesMayo Clinic doesn't have a public directory of staff phone numbers or email addresses. To cont

In [23]:
retriever = db.as_retriever()

In [24]:
# 4. After RAG
print("\n########\nAfter RAG\n")
# after_rag_template = """provide the answer for the question only from this given context:
# {context}
# Question: {question} and only give the answer from the given context
# """
after_rag_template = """Given the context:
{context}

Question: {question}

Please provide all instances of the answer from the given context. If there are multiple instances, list them all.
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)



########
After RAG



In [25]:
chunks = []
async for chunk in after_rag_chain.astream("what are the informations are you able to see in the given context and tell in a brief manner?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 The National Cancer Institute (NCI) manages the National Clinical Trials Network (NCTN), which is a collaborative research network that conducts and supports clinical trials for adult cancers in the United States. The NCTN includes four adult cooperative groups: Eastern Cooperative Oncology Group (ECOG), Southwest Oncology Group (SWOG), North Central Cancer Treatment Group (NCCTG), and the Children's Oncology Group (COG).

The NCTN groups propose concepts for new clinical trials to the NCI Disease/Imaging Steering Committees, which are responsible for evaluating and prioritizing new clinical trials. The committees consist of NCTN group members, disease experts not involved in leadership positions in the groups, biostatisticians, patient advocates, and NCI disease experts.

The overall NCTN budget is $171 million, which allows for an annual enrollment of about 17,000-20,000 participants on cancer treatment and imaging trials. The collaborative approach of the NCTN groups reduces costs 